In [1]:
from qmctorch.scf import Molecule

mol = Molecule(
    atom="Li 0 0 0; H 0 0 3.015",
    unit="bohr",
    calculator="pyscf",
    basis="sto-3g",
    redo_scf=True,
)

INFO:QMCTorch|  ____    __  ______________             _
INFO:QMCTorch| / __ \  /  |/  / ___/_  __/__  ________/ /  
INFO:QMCTorch|/ /_/ / / /|_/ / /__  / / / _ \/ __/ __/ _ \ 
INFO:QMCTorch|\___\_\/_/  /_/\___/ /_/  \___/_/  \__/_//_/ 
INFO:QMCTorch|
INFO:QMCTorch| SCF Calculation
INFO:QMCTorch|  Removing LiH_pyscf_sto-3g.hdf5 and redo SCF calculations
INFO:QMCTorch|  Running scf  calculation
converged SCF energy = -7.86200927212022
INFO:QMCTorch|  Molecule name       : LiH
INFO:QMCTorch|  Number of electrons : 4
INFO:QMCTorch|  SCF calculator      : pyscf
INFO:QMCTorch|  Basis set           : sto-3g
INFO:QMCTorch|  SCF                 : HF
INFO:QMCTorch|  Number of AOs       : 6
INFO:QMCTorch|  Number of MOs       : 6
INFO:QMCTorch|  SCF Energy          : -7.862 Hartree


In [2]:
from qmctorch.wavefunction.orbitals.atomic_orbitals import AtomicOrbitals
from qmctorch.wavefunction.orbitals.atomic_orbitals_backflow import (
    AtomicOrbitalsBackFlow,
)
from qmctorch.wavefunction.orbitals.backflow.backflow_transformation import (
    BackFlowTransformation,
)
from qmctorch.wavefunction.orbitals.backflow.kernels import BackFlowKernelInverse


ao = AtomicOrbitals(mol)

bf = BackFlowTransformation(mol, BackFlowKernelInverse)
aobf = AtomicOrbitalsBackFlow(mol, bf)

Using backend: pytorch


In [23]:
import torch

pos = torch.rand(11, 12)
a, b, c = ao(pos, [0, 1, 2])
print(b.shape)
print(aobf(pos, 1, sum_grad=True).shape)

torch.Size([11, 4, 6, 3])
torch.Size([4, 11, 4, 6])


In [17]:
from qmctorch.wavefunction.slater_jastrow import SlaterJastrow
from qmctorch.wavefunction.jastrows.elec_elec.jastrow_factor_electron_electron import (
    JastrowFactorElectronElectron,
)
from qmctorch.wavefunction.jastrows.elec_elec.kernels import PadeJastrowKernel

jastrow = JastrowFactorElectronElectron(mol, PadeJastrowKernel)

wf = SlaterJastrow(
    mol,
    kinetic="jacobi",
    include_all_mo=True,
    configs="single_double(2,2)",
    jastrow=jastrow,
    backflow=bf,
)

INFO:QMCTorch|
INFO:QMCTorch| Wave Function
INFO:QMCTorch|  Jastrow factor      : True
INFO:QMCTorch|  Jastrow kernel      : ee -> PadeJastrowKernel
INFO:QMCTorch|  Highest MO included : 6
INFO:QMCTorch|  Configurations      : single_double(2,2)
INFO:QMCTorch|  Number of confs     : 4
INFO:QMCTorch|  Kinetic energy      : jacobi
INFO:QMCTorch|  Number var  param   : 66
INFO:QMCTorch|  Cuda support        : False


In [20]:
wf.ao2mo(wf.ao(pos, 1))

2 torch.Size([4, 11, 4, 6])


tensor([[[[-3.4585e+00,  8.2400e-01,  5.9673e-01,  5.2664e-02,  2.4160e-01,
           -3.0029e-01],
          [ 1.1267e-04, -6.3371e-06, -4.4738e-05, -9.9755e-05, -1.3011e-04,
           -5.8372e-06],
          [ 4.8743e-04, -1.6771e-04, -3.6755e-05,  2.5133e-04, -2.0177e-04,
            8.4395e-05],
          [ 5.0144e-03, -1.2489e-03, -8.4502e-04, -1.0444e-04, -5.4762e-04,
            4.2804e-04]],

         [[-8.4607e-01,  2.5296e-01,  1.0549e-01,  1.0515e-01,  1.3094e-01,
           -1.1920e-01],
          [ 2.9163e-04, -1.2779e-04,  8.0072e-06,  1.2990e-04,  1.4239e-05,
            7.4367e-05],
          [ 3.8593e-04, -2.8265e-05, -1.4598e-04, -7.4126e-05, -2.0316e-04,
           -1.2602e-05],
          [ 3.8184e-03, -8.7080e-04, -7.9407e-04,  8.6443e-05, -4.0696e-04,
            2.7453e-04]],

         [[-2.4019e+00,  6.0408e-01,  3.8411e-01,  4.4494e-02,  2.2038e-01,
           -2.4374e-01],
          [ 1.4596e-04, -1.6357e-06, -6.4564e-05, -1.3561e-04, -5.4061e-05,
           

In [27]:
wf.kinetic_energy(pos)

2 torch.Size([11, 4, 6])
2 torch.Size([12, 11, 4, 6])
2 torch.Size([12, 11, 4, 6])


tensor([[ -5.6052],
        [ -6.1737],
        [ -0.9758],
        [  4.1463],
        [-10.8037],
        [  3.9504],
        [ -2.8118],
        [ -7.6169],
        [ -9.7781],
        [ -0.5639],
        [ -1.5769]], grad_fn=<MulBackward0>)